<a href="https://colab.research.google.com/github/skyler14/ICIAR-histology/blob/master/APM_Base_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Version 1 - Base Code to Setup Environment Space on Collab

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
# installing dependencies
import os
import pandas as pd
import numpy as np
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
from pathlib import Path
TRAIN_IMAGE_DIR = Path('/content/drive/My Drive/Projects/iMaterialist/sample/')
ROOT_DIR = Path('/content/')

In [0]:
# # for reference with regards to directory walking
# quake_df = pd.DataFrame(columns=['quakename', 'sensor_id', 'max_sensor'])
# #print(quake_df)

# for paths, dir, files in os.walk('/home/skyler/Downloads/data/csn'):
#     #print(dirnames)
#     for f in files:
#         #print(f)
#         filename = os.path.join(paths,f)
#         #print(os.path.basename(os.path.dirname(filename)))
#         #print(os.path.dirname(filename))
#         path = os.path.dirname(filename)
#         os.chdir(path)
# #       read here

    
    
# #       st = read(f, debug_headers=True)
# #       print(max(st[0].data))
# #       quake_df=quake_df.append({'quakename': os.path.basename(os.path.dirname(filename)), 'sensor_id': f, 'max_sensor': max(st[0].data)-np.median(st[0].data), 'max_clock': np.argmax(st[0].data),'min_sensor': min(st[0].data,)-np.median(st[0].data), 'min_clock': np.argmin(st[0].data),'Latitude':st[0].stats.sac['stla'],'Longitude':st[0].stats.sac['stlo']}, ignore_index=True)
 